<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-[Chapter-08]-텍스트-분석" data-toc-modified-id="1.-[Chapter-08]-텍스트-분석-1">1. [Chapter 08] 텍스트 분석</a></span><ul class="toc-item"><li><span><a href="#1.6-토픽-모델링---20-뉴스그룹" data-toc-modified-id="1.6-토픽-모델링---20-뉴스그룹-1.1">1.6 토픽 모델링 - 20 뉴스그룹</a></span></li><li><span><a href="#1.7-문서-군집화-소개와-실습" data-toc-modified-id="1.7-문서-군집화-소개와-실습-1.2">1.7 문서 군집화 소개와 실습</a></span><ul class="toc-item"><li><span><a href="#1.7.1-Opinion-Review-데이터-세트를-이용한-문서-군집화-수행하기" data-toc-modified-id="1.7.1-Opinion-Review-데이터-세트를-이용한-문서-군집화-수행하기-1.2.1">1.7.1 Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기</a></span></li></ul></li></ul></li></ul></div>

# 1. [Chapter 08] 텍스트 분석

## 1.6 토픽 모델링 - 20 뉴스그룹

- 토픽 모델링(Topic Modeling) : 문서 집합에 숨어 있는 주체를 찾아내는 것 (ex) LSA, LDA)

**LatentDirichletAllocation** 클래스로 LDA 기반의 토픽 모델링 수행. **LDA는 Count 기반의 벡터화만 사용**

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 전자공학, 의학 8개 주제 추출
cats = ['rec.motorcycles','rec.sport.baseball','comp.graphics','comp.windows.x','talk.politics.mideast',\
        'soc.religion.christian','sci.electronics','sci.med']

news_df = fetch_20newsgroups(subset='all', remove=('headers','footers','quotes'), categories=cats, random_state=0)

count_vec = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vec = count_vec.fit_transform(news_df.data)
print('CountVectorizer Shape: ', feat_vec.shape)

CountVectorizer Shape:  (7862, 1000)


In [3]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)   # n_components로 토픽 개수 조정
lda.fit(feat_vec)
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[2.46251560e+02, 1.18842248e+02, 1.51715288e+02, ...,
        1.00147234e+02, 7.63673375e+01, 1.17028758e+02],
       [1.25033020e-01, 1.25052288e-01, 1.25003012e-01, ...,
        1.10644583e+02, 1.51405141e-01, 5.09788954e+01],
       [1.25103419e-01, 1.25075224e-01, 1.25082214e-01, ...,
        6.72008817e+01, 1.25138615e-01, 2.48516614e+00],
       ...,
       [1.05055615e+02, 4.94858011e-01, 2.52075927e+01, ...,
        1.80695744e+01, 1.25115936e-01, 8.33321314e+00],
       [1.25147502e-01, 2.27058083e+02, 5.45176328e+00, ...,
        1.41751120e+00, 7.67217701e+01, 4.49861794e+01],
       [1.25096012e-01, 4.05666840e+00, 1.25049904e-01, ...,
        1.63821915e+02, 1.25049991e-01, 1.49550227e-01]])

components_는 개별 토픽별로 각 word 피처가 얼마나 많이 할당됐는지에 대한 수치를 의미

높은 값일수록 해당 word 피처는 그 토픽의 중심 word가 됨

In [5]:
# 각 토픽별로 연관도가 높은 순으로 word를 나열하는 함수
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic #', topic_idx)
        
        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array 인덱스를 반환
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:no_top_words]
        
        # top_idx 대상인 인덱스별로 feature_names에 해당하는 word feature 추출
        feature_concat = ' '.join([feature_names[i] for i in top_idx])
        print(feature_concat)
        
# CountVectorizer 객체 내의 전체 word의 명칭 추출
feature_names = count_vec.get_feature_names()

display_topics(lda, feature_names, 15)

Topic # 0
year said don didn know game just time went people think did like say home
Topic # 1
god people jesus church think believe christ say does don christian know christians bible faith
Topic # 2
know does thanks like question information help time post advance book just looking group read
Topic # 3
edu com graphics mail ftp information available data pub list computer send software ca 3d
Topic # 4
israel jews jewish israeli dos dos arab turkish people war turkey dos state government greek history
Topic # 5
file image use program window jpeg windows display version color server files using available motif
Topic # 6
armenian armenians people health medical armenia disease turkish patients cancer russian 10 azerbaijan children 92
Topic # 7
like just don ve use good think time know way make used bike want need


명확한 주제어가 추출되지 않고 일반적인 단어가 주를 이루는 Topic 들도 있음

## 1.7 문서 군집화 소개와 실습

- 문서 군집화 : 비슷한 텍스트 구성의 문서를 군집화하는 것. 텍스트 분류 기반의 문서 분류와 달리 비지도학습 기반으로 동작

### 1.7.1 Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review

In [6]:
import pandas as pd
import glob, os

path = r'C:\Users\dalgo\06.Kaggle_Study\data\topics'
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file_ in all_files:
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    # 절대 경로로 주어진 파일명을 가공하고, 맨 마지막 .data 확장자 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    
    # 파일명 list와 파일 내용 list에 파일명과 파일 내용 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())
    
# 파일명 list와 파일 내용 list 객체를 df로 생성
document_df = pd.DataFrame({'filename': filename_list, 'opinion_text': opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [17]:
import string, nltk
from nltk.stem import WordNetLemmatizer 
import re

def ngrams(string, n=2):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [18]:
# 문서를 TF-IDF 형태로 피처 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

tfidf_vec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',\
                            ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vec = tfidf_vec.fit_transform(document_df['opinion_text'])